In [1]:
modelName = 'network_1_100_50_9791.h5'
#modelName = 'network_1_175_75_9848.h5'
#modelName = 'network_1_194_178_9811.h5'
#modelName = 'network_1_100_50_10000.h5'

In [2]:
import pandas as pd
import sys, getopt
import numpy as np
import pprint
import os
import datetime
import glob
import pandas as pd
import emgimporter
import pickle
import time
import importlib


from keras.layers.convolutional import Conv1D
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LocallyConnected2D
from keras.layers import Conv2D, MaxPooling2D, LSTM, Conv1D, MaxPooling1D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model

Using TensorFlow backend.


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [4]:
#modelPath = "/home/myo/models/network_1_100_50_9791.h5"
modelPath = os.path.join("/home/myo/models", modelName)
GPU = False

In [5]:
import numpy as np
import keras
from keras.models import load_model
import os
import sys
import collections, itertools

if not GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

model = load_model(modelPath)

print("loading model: ", modelPath)
print("----------------------------\nModel Summary:")
print(model.summary())
print("----------------------------\n")

loading model:  /home/myo/models/network_1_100_50_9791.h5
----------------------------
Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 1, 50)          450       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 1, 50)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
________________________________________

In [6]:
def ewma(x, alpha):
    '''
    Returns the exponentially weighted moving average of x.

    Parameters:
    -----------
    x : array-like
    alpha : float {0 <= alpha <= 1}

    Returns:
    --------
    ewma : numpy array
        the exponentially weighted moving average
    '''
    # coerce x to an array
    x = np.array(x)
    n = x.size
    # create an initial weight matrix of (1-alpha), and a matrix of powers
    # to raise the weights by
    w0 = np.ones(shape=(n,n)) * (1-alpha) 
    p = np.vstack([np.arange(i,i-n,-1) for i in range(n)])
    # create the weight matrix
    w = np.tril(w0**p,0)
    # calculate the ewma
    return np.dot(w, x[::np.newaxis]) / w.sum(axis=1)

def predict(model,emg):
    return np.squeeze(model.predict(np.expand_dims(np.expand_dims(np.array(emg,ndmin=2),axis=1),axis=-1)))

def movAvg(vec, length=10):
    outvec = list()
    l = len(vec)
    vec2 = [0 for i in range(0,length)]
    vec2.extend(vec)
    for i in range(0,l):
        tmp = np.sum(vec[i-length:i])/length
        outvec.append(tmp)
    return outvec

def getAlphaVec(alpha, length=10):
    stuff = [alpha]
    for i in range(1,length):
        stuff.append(alpha*(1-alpha)**i)
    return stuff

def movExpAvg(vec,alpha=0.2, length=10):
    outvec = list()
    l = len(vec)
    vec2 = [0 for i in range(0,length)]
    vec2.extend(vec)
    
    alphaVec = getAlphaVec(alpha,length)
    
    for i in range(0,l):
        tmp = np.dot( np.array(vec2[i:i+length])[::-1], np.array(alphaVec) )
        outvec.append(tmp)
    return outvec
        
def movP(vec):
    outvec = list()
    l = len(vec)
    vec2 = [0 for i in range(0,length)]
    vec2.extend(vec)
    for i in range(0,l):
        ps = 1 - np.array(vec2[i:i+length])
        tmp = 1- np.prod(ps)
        outvec.append(tmp)
        
    return outvec()

In [7]:
dataPathDict = dict()
dataPathDict["Tobias"] = glob.glob('/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1*')
dataPathDict["Denis"]  = glob.glob('/home/myo/emg_data/ipa_emg/testObj/dynamicsDenisS1_*')
dataPathDict["Daniel"] = glob.glob('/home/myo/emg_data/ipa_emg/testObj/dynamicsS2_*')

print(dataPathDict)
for dat in dataPathDict:
    tstampList = list(set([k.split('-',1)[-1] for k in dat]))
    tstampList.sort()

{'Tobias': ['/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1imu-20170908-142326', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1imu-20170908-142233', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1aux-20170908-141647', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1-20170908-141601', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1-20170908-141943', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1imu-20170908-141745', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1imu-20170908-142418', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1imu-20170908-142459', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1-20170908-142233', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1-20170908-142133', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1-20170908-141647', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1imu-20170908-141647', '/home/myo/emg_data/ipa_emg/testObj/dynamicsTobiasS1_1aux-20170908-142041', '/home/myo/emg_

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column, row
from bokeh.palettes import Spectral11, Category20, Set1, Reds
from bokeh.models import Range1d
from bokeh.models import LinearAxis, Range1d
from bokeh.charts import TimeSeries, output_notebook, output_file, reset_output

for dataPathKey in dataPathDict:
    dataPathList = dataPathDict[dataPathKey]
    reset_output()
    outputName = str(dataPathKey) + "_plot.html"
    output_file(outputName, title="Plotting all Experiments")

    plotlist = list()
    alphaList = [0.2]
    palette = Reds[9]

    tstampList = list(set([k.split('-',1)[-1] for k in dataPathList]))
    tstampList.sort()
    for tstamp in tstampList:

        krappl = dict()
        for fName in dataPathList:
            if tstamp in fName:
                if 'imu' in fName:
                    krappl['imu'] = fName
                elif 'aux' in fName:
                    krappl['aux'] = fName
                else:
                    krappl['norm'] = fName

        data = list()
        data.append( pd.read_csv(krappl['aux'],sep='\t') )
        data.append( pd.read_csv(krappl['norm'],sep='\t'))

        kk = list()
        for i,r in data[1].iterrows():
            tmp = 0
            tmp += predict(model,np.array(r[1:]))[1]
            kk.append(tmp)

        data[0]["t"]

        comVolt = data[0]["V0"] + data[0]["V1"] + data[0]["V2"]
        comVolt_norm = (comVolt - comVolt.min())/(comVolt.max()-comVolt.min())
        comEMG = data[1]["EMG0"]
        for i in range(1,8):
            name = "EMG" + str(i)
            comEMG  += data[1][name]

        comEMG_norm = (comEMG - comEMG.min())/(comEMG.max() - comEMG.min())

        p = figure(x_axis_type="datetime",plot_width=1600,plot_height=600,x_axis_label="Time", y_axis_label = "Normed Combined Voltage",title='') # str(datetime.datetime.strptime(tstamp,'%Y%m%d-%H%M%S'))
        
        p.y_range=Range1d(0,1)

        t_offset = min(data[0]["t"].min(),data[1]["t"].min())

        p.line(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in data[0]["t"]],y = comVolt_norm,line_width=3, legend="Combined Voltage")
        p.line(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in data[1]["t"]],y = comEMG_norm,color='green', legend="Combined EMG Activation")
        for i,alpha in enumerate(alphaList):
            p.line(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in data[1]["t"]],
                   y = [ ((int(i>0.6)+int(i>0.4))*0.5)*0.8 + 0.1 for i in movExpAvg(kk,alpha,100)],color=palette[i],line_width=5-i, legend="Filtered grip probability with alpha " + str(alpha))
            
         # Formating stuff
        p.xaxis.axis_label_text_font_size = "25pt"
        p.xaxis.axis_label_text_font = 'helvetica'

        p.yaxis.axis_label_text_font_size = "25pt"
        p.yaxis.axis_label_text_font = 'helvetica'
        
        p.yaxis.major_label_text_font_size = "19pt"
        p.yaxis.major_label_text_font = 'helvetica'

        p.xaxis.major_label_text_font_size = "19pt"
        p.xaxis.major_label_text_font = 'helvetica'
        
        p.legend.label_text_font_size = "19pt"
        p.legend.label_text_font = 'helvetica'
        
        p.legend.location = "top_left"
        
        # End Formatingn stuff

        plotlist.append(p)
        print('plotted ' +str(datetime.datetime.strptime(tstamp,'%Y%m%d-%H%M%S')))

    show(column(plotlist))
    print('done with ' + outputName)

In [19]:
def norm(l):
    l =  np.abs((l - np.mean(l)))
    return (l/(l.max() - l.min()))

In [20]:
dataPathList2 = glob.glob('/home/myo/emg_data/ipa_emg/testObj/Sync8_8*')

In [21]:
dataPathList2

['/home/myo/emg_data/ipa_emg/testObj/Sync8_8imu-20170905-133114',
 '/home/myo/emg_data/ipa_emg/testObj/Sync8_8-20170905-133114',
 '/home/myo/emg_data/ipa_emg/testObj/Sync8_8aux-20170905-133114']

In [22]:
data2 = list()
for fName in dataPathList2:
    data2.append( pd.read_csv(fName,sep='\t') )

In [23]:
list(data2[0])

['t',
 'QX_IMU',
 'QY_IMU',
 'QZ_IMU',
 'QW_IMU',
 'ACCX',
 'ACCY',
 'ACCZ',
 'GX',
 'GY',
 'GZ']

In [24]:
for i in data2:
    if "ACCX_EXT" in list(i):
        t_ext = i["t"]
        accx_ext = norm(i["ACCX_EXT"])
        accy_ext = norm(i["ACCY_EXT"])
        accz_ext = pd.rolling_mean(norm(i["ACCZ_EXT"]),20*5)
    elif "ACCX" in list(i):
        t = i["t"]
        accx = norm(i["ACCX"])
        accy = norm(i["ACCY"])
        accz = pd.rolling_mean(norm(i["ACCZ"]),5)


/home/myo/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
  # This is added back by InteractiveShellApp.init_path()
/home/myo/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=100,center=False).mean()
  


In [25]:
print(len(accy), len(accz))

2345 2345


In [58]:
from scipy import signal
reset_output()
output_file("lines.html", title="line plot example")
palette = Category20[12]
paletteX = palette[0:3]
paletteY = palette[4:7]
paletteZ = palette[8:11]
t_offset = min(t.min(),t_ext.min())


p2 = figure(x_axis_type="datetime",plot_width=1600,plot_height=800,x_axis_label="Time [s]", y_axis_label = "Acceleration in Z direction [g]")
#p2.line(x = [datetime.datetime.fromtimestamp(ts) for ts in t],y = accx,line_width=3, color=palette[1], legend="X")
#p2.line(x = [datetime.datetime.fromtimestamp(ts) for ts in t],y = accy,line_width=3, color=palette[3], legend="Y")
p2.line(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in t],y = accz,line_width=5, color=palette[5], legend="Bracelet Sensor")

#p2.line(x = [datetime.datetime.fromtimestamp(ts) for ts in t_ext],y = accx_ext,line_width=3,color=palette[7], legend="X_EXT")
#p2.line(x = [datetime.datetime.fromtimestamp(ts) for ts in t_ext],y = accy_ext,line_width=3,color=palette[9], legend="Y_EXT")
p2.line(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in t_ext],y = accz_ext,line_width=2,color=palette[11], legend="External Sensor")

show(p2)

from detect_peaks import detect_peaks
import peakdetect

extMpd = 10
extMph = None
extThr = 0.0001

Mpd = 10
Mph = None
Thr = 0.001

#x_ext = detect_peaks(accx_ext,show=False, mpd=extMpd, mph=extMph, threshold=extThr, valley=False) # 
#y_ext = detect_peaks(accy_ext,show=False, mpd=extMpd, mph=extMph, threshold=extThr, valley=False) #
#z_ext1 = detect_peaks(accz_ext,show=False, mpd=extMpd, mph=extMph, threshold=extThr, valley=False) #
t1,t2 = peakdetect.peakdetect(accz_ext,lookahead=50*20,delta=0.1)
#t1,t2 = peakdetect.peakdetect(accz_ext,t_ext)
z_ext2_maximum = [t[0] for t in t1]
z_ext2_minimum = [t[0] for t in t2]
t1.extend(t2)
z_ext2 = [t[0] for t in t1 ]

print(len(z_ext2))
#x     = detect_peaks(accx    ,show=False, mpd=Mpd, mph=Mph, threshold=Thr, valley=False) # 
#y     = detect_peaks(accy    ,show=False, mpd=Mpd, mph=Mph, threshold=Thr, valley=False) # 
#z1     = detect_peaks(accz    ,show=False, mpd=Mpd, mph=Mph, threshold=Thr, valley=False) #
t1,t2  = peakdetect.peakdetect(accz,lookahead=50,delta=0.1)
#t1,t2 = peakdetect.peakdetect(accz,t)
z2_maximum = [t[0] for t in t1]
z2_minimum = [t[0] for t in t2]
t1.extend(t2)
z2 = [t[0] for t in t1 ]


print(len(z2))

s = 15
#p2.circle(x = [datetime.datetime.fromtimestamp(ts) for ts in t_ext[x_ext] ], y = [k for k in accx_ext[x_ext]],color=palette[6])
#p2.square(x = [datetime.datetime.fromtimestamp(ts) for ts in t_ext[z_ext2_minimum] ], y = [k for k in accz_ext[z_ext2_minimum]],color=palette[8], legend="Z_Ext Minimum", size=s)
p2.cross(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in t_ext[z_ext2_maximum] ], y = [k for k in accz_ext[z_ext2_maximum]],color=palette[10], legend= "External Sensor Maximum", size=s, line_width= 5)
#p2.circle(x = [datetime.datetime.fromtimestamp(ts) for ts in t_ext[z_ext1] ], y = [k for k in accz_ext[z_ext1]],color=palette[10])

#p2.circle(x = [datetime.datetime.fromtimestamp(ts) for ts in t[x] ], y = [k for k in accx[x]], color=palette[0] )
#p2.circle(x = [datetime.datetime.fromtimestamp(ts) for ts in t[z2_minimum] ], y = [k for k in accz[z2_minimum]], color=palette[2] ,legend="Z Minimum",size=s)
p2.triangle(x = [datetime.datetime.fromtimestamp(ts)-datetime.datetime.fromtimestamp(t_offset) for ts in t[z2_maximum] ], y = [k for k in accz[z2_maximum]], color=palette[4], legend="Bracelet Maximum",size=s )
#p2.circle(x = [datetime.datetime.fromtimestamp(ts) for ts in t[z1] ], y = [k for k in accz[z1]], color=palette[4] )


p2.xaxis.axis_label_text_font_size = "25pt"
p2.xaxis.axis_label_text_font = 'helvetica'


p2.yaxis.axis_label_text_font_size = "25pt"
p2.yaxis.axis_label_text_font = 'helvetica'


p2.yaxis.major_label_text_font_size = "19pt"
p2.yaxis.major_label_text_font = 'helvetica'


p2.xaxis.major_label_text_font_size = "19pt"
p2.xaxis.major_label_text_font = 'helvetica'


p2.title.text_font_size = '13pt'
p2.title.text_font = 'helvetica'

p2.legend.label_text_font_size= "19pt"
p2.legend.label_text_font = 'helvetica'
p2.legend.location = "top_left"

show(p2)
print("done")

13
13
done


In [59]:
exts = [z_ext2_maximum, z_ext2_minimum]
normal = [z2_maximum, z2_minimum]

In [60]:
def list_matching(list1, list2):
    list1_copy = list1[:]
    pairs = []
    for i, e in enumerate(list2):
        elem = find_closest(list1_copy, e)
        pairs.append([i, list1.index(elem)])
        list1_copy.remove(elem)
    return pairs

def find_closest(alist, target):
    return min(alist, key=lambda x:abs(x-target))

In [61]:
blub = [0,0,0]
blublen = [0,0,0]

for number, stuff in enumerate(exts):
    times_armband = list(t[normal[number]])
    times_extern = list(t_ext[exts[number]])

    if len(times_armband) > len(times_extern):
        pairs = list_matching(times_armband,times_extern)
        indexes = [1,0]
    else:
        pairs = list_matching(times_extern,times_armband)
        indexes = [0,1]

    
    for i in pairs:
        blub[number] += (times_armband[i[indexes[0]]] - times_extern[i[indexes[1]]])**2
    blublen[number] += len(pairs)


print(blub)
print(blublen)
total_delay = (blub[0])/(blublen[0] -1)
print(total_delay)

[0.0024789460787246753, 37.961282841423667, 0]
[7, 6, 0]
0.000413157679787


In [62]:
datetime.timedelta(seconds=total_delay).microseconds/1000

0.413